<a href="https://colab.research.google.com/github/CameronKenworthyCode/python/blob/main/PLSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Built T-R-S map data by querying BLM public land survey**

#Setup

In [ ]:
!pip install requests
!pip install bs4
!pip install html2text
!pip install -U -q PyDrive

import html2text
from ast import literal_eval
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import random
import json

auth.authenticate_user() #establish connection to a gdrive to write the eventual csv file to
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1DWFETkLt-dWcA7CfymejJe7O-UJxSR4B' in parents and trashed=False"}).GetList() 

EWA_TRS = pd.read_csv('drive/My Drive/NEWICC/PLSS/EWA_trs.csv', index_col=0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive/


# TODO

~~- generate URLS~~

~~- batch query~~
~~- save geometry long/lat coords~~

  ~~- number of coords given varies per section~~
    
    - get corners
      - get all points along each edge
    - infer outline of township range from edges of sections
- infer shape from coords
- save all as csv
- code to calculate if coord is in shape

#Functions

In [ ]:
# generate set of strings in form WA330##0N0##0E0SN##0 or WA330##0N0##0W0SN##0
# return list of these strings of possible sections 1-60 for each T/R pair
def generate_url(town, _range):
  t_ = town[:-1]
  if int(t_) < 10:
    t_ = '0' + t_
  r_ = _range[:-1]
  if int(r_) < 10:
    r_ = '0' + r_
  r_ = r_ + '0' + _range[-1:]
  base_string = 'WA330'+ t_ + '0N0' + r_ + '0SN'
  section_list = []
  for i in range(1, 60):
    s_ = str(i) + '0'
    if i < 10:
      s_ = '0' + str(i) + '0'
    section_list.append(base_string+s_)
  
  return section_list

#Generate URL Queries

T/R pairs in Washington State:

TOWN: 1N -> 41N

RANGE: 16W -> 47E


In [51]:
# generate urls in batches of 25 queries
trs = []

# for all T/R where R is West
for i in range(1,42):
  for j in range(1,17):
    trs.append([str(i)+'N', str(j)+'W'])

# for all T/R where R is East
for i in range(1, 42):
  for j in range(1, 48):
    trs.append([str(i)+'N', str(j)+'E'])

# All the section url words for each T/R
url_dict = {}
for pair in trs:
  key = pair[0] + '-' + pair[1]
  value = generate_url(pair[0], pair[1])
  url_dict[key] = value

#strings to build url query
base_url = 'https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs='
built_url = ''
base_url_end = '&returnalllevels=&f=pjson'

# go through each T/R pair, and take the next 25 section urls to build a query, save the results and continue to the next 25
count = 0
keys = {} # for building PLSS_dict
urls = [] # for sending batches to requests.get()
for i in range(len(trs)):
  # for each T/R pair
  for j in range(0, 59):
    # for each possible section in each T/R pair
    if count == 25: # url query maxed, store and refresh
      urls.append(base_url+built_url+base_url_end)
      built_url = ''
      count = 0
    count += 1
    built_url = built_url + url_dict[trs[i][0] + '-' + trs[i][1]][j] + '+%7C+'
    # create dict of WA330##0N0##0E0SN##0 keys corresponding to regular T/R/S notation
    # use dict to correctly put away coordinates to correct T/R/S in PLSS_dict
    keys[url_dict[trs[i][0] + '-' + trs[i][1]][j]] = trs[i][0] + '-' + trs[i][1] + '-' + str(j+1) 

# Query gis.blm.gov for T/R/S coordinate shapes

In [ ]:
'''

ALREADY RAN
TAKES A LONG TIME
JUST LOAD CSV INSTEAD

'''
# fill dict with coordinate outlines of each section in every T/R
# by querying gis.blm.gov

PLSS_dict_raw = {}

for i in range(len(urls)):
  url = urls[i]
  parser = html2text.HTML2Text()
  parser.ignore_links = True
  result = requests.get(url)
  src = result.content
  soup = BeautifulSoup(src, 'lxml')
  result = json.loads(soup.text)
  print(str(round(i/len(urls)*100, 3)) + ': ' + str(len(result['features'])))
  for i in range(len(result['features'])):
    #{'TT-RR-SS': [[List of section defining coordinates]]}
    PLSS_dict_raw[keys[result['features'][i]['attributes']['landdescription']]] = result['features'][i]['geometry']['rings'][0]

plss_df = pd.DataFrame({ key:pd.Series(value) for key, value in PLSS_dict_raw.items() })
plss_df = plss_df.T
plss_df.to_csv('plss.csv')
!cp plss.csv "drive/My Drive/NEWICC/PLSS/"

# lookup table for easy search
tr_index = {}

# for each valid T/R/S word
# create a row of their integer T/R/S values
for index in plss_df.index:
  trs_list = index.split('-')
  if trs_list[1][-1:] == 'W':
    continue
  town = int(trs_list[0][:-1])
  range = int(trs_list[1][:-1])
  section = int(trs_list[2])
  tr_index[index] = {"town": town, 'range':range, 'section':section}

# build final dataframe
tr_index_df = pd.DataFrame(tr_index).T
trs_df = tr_index_df.join(plss_df)
trs_df.to_csv('EWA_trs.csv')
!cp EWA_trs.csv "drive/My Drive/NEWICC/PLSS/"